# SPWD algorithm usability for the CQM hybrid solver (Improving CQMcapacity with SPWD)

For the explanation and results of this experiment please refer to [this context](https://github.com/mkroczek/SPWD-experiments/tree/master/experiments/cqm_scheduling).

## How to run
1. clone https://github.com/wfcommons/pegasus-instances repository. Workflows from this repository will be used in the experiment.
2. set PEGASUS_INSTANCES_DIR to the directory containing cloned repo
3. remember to have DWAVE_API_TOKEN env variable to be able to use CQM
4. remember to uncomment code "Enable CQM for final experiment run"

In [1]:
PEGASUS_INSTANCES_DIR = "/Users/marcinkroczek/code/pegasus-instances"

In [2]:
import sys
import os

sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../../"))

In [3]:
from abc import ABC, abstractmethod
from dataclasses import dataclass

from report import ExecutionReport, Solution
from QHyper.problems.workflow_scheduling import Workflow
from decomposition.qhyper.algorithm import WorkflowDecompositionQHyperAdapter
from decomposition.qhyper.solver import DecomposedWorkflowSchedulingSolver, WorkflowSchedulingSolverDecorator, WorkflowSchedule
from decomposition.qhyper.problem import WorkflowSchedulingOneHotEnhanced
from QHyper.solvers.classical.gurobi import Gurobi
from QHyper.solvers.quantum_annealing.dwave.cqm import CQM

In [6]:
class ExperimentResult(ABC):
    @abstractmethod
    def plot(self):
        pass

@dataclass
class AlgorithmRun:
    max_subgraph_size: int
    decomposition_schedule: WorkflowSchedule
    reference_schedule: WorkflowSchedule

class SolverFactory(ABC):
    def __init__(self, tasks_file, machines_file, deadline):
        self.tasks_file = tasks_file
        self.machines_file = machines_file
        self.deadline = deadline
    
    @abstractmethod
    def get_decomposed_solver(self, max_subgraph_size: int):
        pass
    
    @abstractmethod
    def get_reference_solver(self):
        pass
    
class GurobiSolverFactory(SolverFactory):
    def get_decomposed_solver(self, max_subgraph_size: int):
        workflow = Workflow(self.tasks_file, self.machines_file, self.deadline)
        division = WorkflowDecompositionQHyperAdapter(workflow).decompose(max_subgraph_size)
        problems = map(lambda w: WorkflowSchedulingOneHotEnhanced(w), division.workflows)
        solvers = map(lambda p: WorkflowSchedulingSolverDecorator(Gurobi(p)), problems)
        return DecomposedWorkflowSchedulingSolver(list(solvers), division)

    def get_reference_solver(self):
        workflow = Workflow(self.tasks_file, self.machines_file, self.deadline)
        return WorkflowSchedulingSolverDecorator(Gurobi(WorkflowSchedulingOneHotEnhanced(workflow)))

class CQMSolverFactory(SolverFactory):
    def __init__(self, tasks_file, machines_file, deadline, time=5):
        super().__init__(tasks_file, machines_file, deadline)
        # time sets a maximum execution time for the experiment on CQM. It is expressed in seconds
        self.time=time

    def get_decomposed_solver(self, max_subgraph_size: int):
        workflow = Workflow(self.tasks_file, self.machines_file, self.deadline)
        division = WorkflowDecompositionQHyperAdapter(workflow).decompose(max_subgraph_size)
        problems = map(lambda w: WorkflowSchedulingOneHotEnhanced(w), division.workflows)
        solvers = map(lambda p: WorkflowSchedulingSolverDecorator(CQM(problem=p, time=self.time)), problems)
        return DecomposedWorkflowSchedulingSolver(list(solvers), division)

    def get_reference_solver(self):
        workflow = Workflow(self.tasks_file, self.machines_file, self.deadline)
        return WorkflowSchedulingSolverDecorator(CQM(problem=WorkflowSchedulingOneHotEnhanced(workflow), time=self.time))


class Experiment(ABC):
    def __init__(self, tasks_file, machines_file, deadline):
        self.tasks_file = tasks_file
        self.machines_file = machines_file
        self.deadline = deadline


    @abstractmethod
    def run(self) -> ExperimentResult:
        pass


class CQMDecompositionExperiment(Experiment):
    class CQMExperimentResult(ExperimentResult):
        def __init__(
            self, 
            gurobi_scheduling: AlgorithmRun, 
            cqm_scheduling: AlgorithmRun,
            tasks_file: str,
            machines_file: str,
            deadline: int,
            max_subgraph_size: int
        ):
            self.gurobi_scheduling: AlgorithmRun = gurobi_scheduling
            self.cqm_scheduling: AlgorithmRun = cqm_scheduling
            self.tasks_file=tasks_file
            self.machines_file=machines_file
            self.deadline=deadline
            self.max_subgraph_size=max_subgraph_size

        def save_execution_report(self, save_dir: str, file_prefix: str):
            # 
            # cqm_decomposed_report = self._get_execution_report(
            #     solver="CQM",
            #     workflow_schedule=self.cqm_scheduling.decomposition_schedule
            # )
            # cqm_decomposed_report.write_json(os.path.join(save_dir, f"{file_prefix}_cqm_decomposed.json"))

            gurobi_decomposed_report = self._get_execution_report(
                solver="Gurobi",
                workflow_schedule=self.gurobi_scheduling.decomposition_schedule
            )
            gurobi_decomposed_report.write_json(os.path.join(save_dir, f"{file_prefix}_gurobi_decomposed.json"))

            gurobi_raw_report = self._get_execution_report(
                solver="Gurobi",
                workflow_schedule=self.gurobi_scheduling.reference_schedule
            )
            gurobi_raw_report.write_json(os.path.join(save_dir, f"{file_prefix}_gurobi_raw.json"))

        
        def _get_execution_report(self, solver: str, workflow_schedule: WorkflowSchedule):
            return ExecutionReport(
                workflow_file=self.tasks_file,
                machines_file=self.machines_file,
                deadline=self.deadline,
                max_subgraph_size=self.max_subgraph_size,
                solver=solver,
                solution=Solution.from_workflow_schedule(workflow_schedule)
            )
        
        def plot(self):
            pass
        
    def __init__(self, tasks_file, machines_file, max_subgraph_size: int):
        super().__init__(tasks_file, machines_file, deadline_as_cpv(tasks_file, machines_file))
        self.max_subgraph_size: int = max_subgraph_size
        self.gurobi_factory: SolverFactory = GurobiSolverFactory(self.tasks_file, self.machines_file, self.deadline)
        self.cqm_factory: SolverFactory = CQMSolverFactory(self.tasks_file, self.machines_file, self.deadline)

    def run(self) -> CQMExperimentResult:
        gurobi_run = AlgorithmRun(
            self.max_subgraph_size, 
            self.gurobi_factory.get_decomposed_solver(self.max_subgraph_size).solve(), 
            self.gurobi_factory.get_reference_solver().solve()
        )
        # Enable CQM for final experiment run
        # cqm_run = AlgorithmRun(
        #     self.max_subgraph_size,
        #     self.cqm_factory.get_decomposed_solver(self.max_subgraph_size).solve(),
        #     None
        # )
        cqm_run = None
        return self.CQMExperimentResult(
            gurobi_run, 
            cqm_run,
            self.tasks_file,
            self.machines_file,
            self.deadline,
            self.max_subgraph_size
        )

def deadline_as_cpv(tasks_file, machines_file):
    workflow = Workflow(tasks_file, machines_file, 100000)
    mean_times = workflow.time_matrix.mean(axis=1).to_dict()

    def path_load(p):
        return sum([mean_times[t] for t in p])

    return int(max(path_load(p) for p in workflow.paths))

# Actual experiment runs

### Montage 310 nodes, mss = 100

In [ ]:
montage_310 = f"{PEGASUS_INSTANCES_DIR}/montage/chameleon-cloud/montage-chameleon-2mass-015d-001.json"
machines_file = "../resources/machines/linear_smaller_diff.json"
max_subgraph_size = 100

experiment = CQMDecompositionExperiment(montage_310, machines_file, max_subgraph_size)
experiment_result = experiment.run()
experiment_result.save_execution_report(save_dir = "./montage_310_100", file_prefix = "montage_310_mss_100")

### Montage 472, mss = 150

In [ ]:
montage_472 = f"{PEGASUS_INSTANCES_DIR}/montage/chameleon-cloud/montage-chameleon-dss-10d-001.json"
machines_file = "../resources/machines/linear_smaller_diff.json"
max_subgraph_size = 150

experiment = CQMDecompositionExperiment(montage_472, machines_file, max_subgraph_size)
experiment_result = experiment.run()
experiment_result.save_execution_report(save_dir = "./montage_472_150", file_prefix = "montage_472_mss_150")

### Montage 619, mss = 200

In [ ]:
montage_619 = f"{PEGASUS_INSTANCES_DIR}/montage/chameleon-cloud/montage-chameleon-2mass-025d-001.json"
machines_file = "../resources/machines/linear_smaller_diff.json"
max_subgraph_size = 200

experiment = CQMDecompositionExperiment(montage_619, machines_file, max_subgraph_size)
experiment_result = experiment.run()
experiment_result.save_execution_report(save_dir = "./montage_619_200", file_prefix = "montage_619_mss_200")

### Montage 1066 nodes, mss = 350

In [ ]:
montage_1066 = f"{PEGASUS_INSTANCES_DIR}/montage/chameleon-cloud/montage-chameleon-dss-125d-001.json"
machines_file = "../resources/machines/linear_smaller_diff.json"
max_subgraph_size = 350

experiment = CQMDecompositionExperiment(montage_1066, machines_file, max_subgraph_size)
experiment_result = experiment.run()
experiment_result.save_execution_report(save_dir = "./montage_1066_350", file_prefix = "montage_1066_mss_350")

Small test

In [8]:
genome_small = f"{PEGASUS_INSTANCES_DIR}/1000genome/chameleon-cloud/1000genome-chameleon-2ch-250k-001.json"
machines_file = "../resources/machines/linear_smaller_diff.json"
max_subgraph_size = 30

experiment = CQMDecompositionExperiment(genome_small, machines_file, max_subgraph_size)
experiment_result = experiment.run()
experiment_result.save_execution_report(save_dir = "./genome", file_prefix = "genome_small")